# Initialisation

In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2regional as br
import sqlite3
import pandas as pd
import geopandas as gp
import numpy as np
import country_converter as coco
import logging
bd.projects.set_current("Brightway regional tutorial")

# Importing regionalized CF for freshwater acidification

## Creating the map of the cfs at the country scale

### Function for tidying dataframe

In [2]:
def clean_up_dataframe(df):
    # remove duplicates
    df = df.drop_duplicates()
    # fix index
    df = df.reset_index().drop('index', axis=1)
    return df

### Create a connection with IW+ native resolution CF

In [3]:
dbfile = "C:/Impact_world_+_2_1/impact_world_plus_2.1_source.db"

In [4]:
con = sqlite3.connect(dbfile)

### Loading data

In [5]:
cell_emissions = {
'NH3': pd.read_sql('SELECT * FROM [CF - regionalized - AcidFW - native (NH3 emissions to air)]',con).set_index('cell'),
'NOx': pd.read_sql('SELECT * FROM [CF - regionalized - AcidFW - native (NOx emissions to air)]',con).set_index('cell'),
'SO2': pd.read_sql('SELECT * FROM [CF - regionalized - AcidFW - native (SO2 emissions to air)]',con).set_index('cell')}

inter_country = pd.read_sql('SELECT * FROM [SI - Acidification/Eutrophication - Countries cell resolution]',con).set_index('cell').T
inter_continent = pd.read_sql('SELECT * FROM [SI - Acidification/Eutrophication - Continents cell resolution]',con).set_index('cell').T

### Calculation midpoint and damage CF

Calculation of the emissions of acid products per countries

In [6]:
country_emissions = {}
continent_emissions = {}

In [7]:
for substance in cell_emissions:
    cell_emissions[substance].index.name = None
    country_emissions[substance] = inter_country.dot(cell_emissions[substance].Emission)
    continent_emissions[substance] = inter_continent.dot(cell_emissions[substance].Emission)

Initialization of the midpoint cf dataframe

In [8]:
cfs_midpoint = pd.DataFrame(None, cell_emissions.keys(), country_emissions['SO2'].index)

Weighted means of midpoints at different scales (country, continents and glo)

In [9]:
for substance in cell_emissions:
            # for countries
            for country in country_emissions[substance].index:
                try:
                    aggregated_cf = (cell_emissions[substance].loc[:, 'midpoint'] *
                                     cell_emissions[substance].loc[:, 'Emission'] /
                                     country_emissions[substance].loc[country] *
                                     inter_country.loc[country]).sum()
                    cfs_midpoint.loc[substance, country] = aggregated_cf
                except ZeroDivisionError:
                    pass

            # for continents
            for continent in continent_emissions[substance].index:
                try:
                    aggregated_cf = (cell_emissions[substance].loc[:, 'midpoint'] *
                                     cell_emissions[substance].loc[:, 'Emission'] /
                                     continent_emissions[substance].loc[continent] *
                                     inter_continent.loc[continent]).sum()
                    cfs_midpoint.loc[substance, continent] = aggregated_cf
                except ZeroDivisionError:
                    pass

            # global
            GLO_value = (cell_emissions[substance].loc[:, 'midpoint'] *
                         cell_emissions[substance].loc[:, 'Emission'] /
                         cell_emissions[substance].Emission.sum()).sum()
            cfs_midpoint.loc[substance, 'GLO'] = GLO_value
# divide by reference flow -> SO2
cfs_midpoint /= cfs_midpoint.loc['SO2', 'GLO']

In [10]:
cfs_midpoint

,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,Antigua and Barbuda,Argentina,...,Yemen,Zambia,Zimbabwe,RNA,RLA,RER,RAS,RAF,OCE,GLO
NH3,0.19341,0.209535,0.131106,0.004133,0.180141,0.013075,0.201398,0.006115,0.181716,0.214627,...,0.066913,0.017845,0.032775,7.020598,0.200554,0.597229,0.180494,0.048254,0.014288,0.694883
NOx,0.085481,0.107495,0.053105,0.006244,0.102849,0.003654,0.056765,0.019462,0.044656,0.120129,...,0.011971,0.006458,0.013023,1.602667,0.078563,0.292908,0.106524,0.013184,0.004150,0.393038
SO2,0.227948,0.237582,0.146309,0.005854,0.304463,0.024613,0.134264,0.002865,0.129912,0.244792,...,0.026665,0.025577,0.122471,4.811017,0.231286,1.013358,0.727353,0.157147,0.021855,1.000000


In [11]:
# Saving countries name for later in order to see which one do not have ISO2 code
countries_name = cfs_midpoint.columns

### Data formatting

Converting country names to ISO2 code

In [12]:
# lines of logger to avoid having warnings showing up
coco_logger = coco.logging.getLogger()
coco_logger.setLevel(coco.logging.CRITICAL)
cfs_midpoint.columns = coco.convert(cfs_midpoint.iloc[:, :-7], to='ISO2') + ['RNA', 'RLA', 'RER', 'RAS', 'RAF','OCE', 'GLO']

Country with no ISO2 code

In [13]:
countries_name[cfs_midpoint.columns.get_loc('not found')]

Index(['Glorioso Island', 'Jan Mayen', 'Jarvis Is', 'Johnston Atoll',
       'Juan De Nova Island', 'Midway Is', 'Netherlands Antilles',
       'Virgin Islands'],
      dtype='object')

Droping the countries with no ISO code

In [14]:
cfs_midpoint.drop('not found', axis=1, inplace=True)

Dataframe for the import in brightway

In [15]:
cfs_midpoint = cfs_midpoint.stack().reset_index()

In [16]:
cfs_midpoint.columns = ['Elem flow', 'Region code', 'CF value']
cfs_midpoint.loc[:, 'CF unit'] = 'kg SO2 eq'
cfs_midpoint.loc[:, 'MP or Damage'] = 'Midpoint'
cfs_midpoint.loc[:, 'Impact category'] = 'Freshwater acidification'
cfs_midpoint.loc[:, 'Compartment'] = 'Air'
cfs_midpoint.loc[:, 'Sub-compartment'] = '(unspecified)'
cfs_midpoint.loc[:, 'Elem flow unit'] = 'kg'
cfs_midpoint.loc[:, 'Native geographical resolution scale'] = 'Country'
cfs_midpoint.loc[cfs_midpoint.loc[:, 'Elem flow'] == 'NH3', 'CAS number'] = '7664-41-7'
cfs_midpoint.loc[cfs_midpoint.loc[:, 'Elem flow'] == 'NOx', 'CAS number'] = '11104-93-1'
cfs_midpoint.loc[cfs_midpoint.loc[:, 'Elem flow'] == 'SO2', 'CAS number'] = '7446-09-5'

In [17]:
cfs_midpoint = clean_up_dataframe(cfs_midpoint)

In [18]:
cfs_midpoint.loc[[i for i in cfs_midpoint.index if cfs_midpoint.loc[i, 'Region code'] in ['RNA', 'RLA', 'RER', 'RAS', 'RAF', 'OCE', 'GLO']],'Native geographical resolution scale'] = 'Continent'
cfs_midpoint.loc[cfs_midpoint.loc[:, 'Region code'] == 'GLO', 'Native geographical resolution scale'] = 'Global'

### Applying Stoechiometric ratios

In [19]:
stoc = pd.read_sql('SELECT * FROM [SI - Stoechiometry]', con)
stoc = stoc.loc[stoc.loc[:, 'Impact category'] == 'Freshwater acidification']
for i in stoc.index:
    proxy = stoc.loc[i, 'Proxy molecule']
    comp = stoc.loc[i, 'Compartment']
    df = cfs_midpoint[cfs_midpoint.loc[:, 'Elem flow'] == proxy].loc[cfs_midpoint.loc[:, 'Compartment'] == comp].copy('deep')
    if not df.empty:
        df.loc[:, 'Elem flow'] = stoc.loc[i, 'Elem flow name']
        df.loc[:, 'CAS number'] = stoc.loc[i, 'CAS number']
        df.loc[:, 'CF value'] *= stoc.loc[i, 'Proxy ratio']
        cfs_midpoint = clean_up_dataframe(pd.concat([cfs_midpoint, df]))

In [20]:
cfs_midpoint = cfs_midpoint.drop(cfs_midpoint.loc[cfs_midpoint.loc[:, 'Elem flow'].isin(['NH3', 'NOx', 'SO2'])].index)

Dropping the regions CF - only countries CF would be used at this stage

In [21]:
cfs_midpoint = cfs_midpoint.drop(cfs_midpoint.loc[cfs_midpoint.loc[:, 'Region code'].isin(['RNA', 'RLA', 'RER', 'RAS', 'RAF','OCE'])].index)

Dropping unecessary columns from the dataframe

In [22]:
cfs_midpoint = cfs_midpoint.drop( labels = ["CF unit", "MP or Damage", "Impact category","Elem flow unit","CAS number","Native geographical resolution scale"],axis=1)

Saving the GLO Cfs in another dataframe

In [23]:
cfs_glo = cfs_midpoint.loc[cfs_midpoint.loc[:, 'Region code'].isin(['GLO'])]

In [24]:
cfs_glo 

,Elem flow,Region code,CF value,Compartment,Sub-compartment
935,Ammonia,GLO,0.694883,Air,(unspecified)
1169,"Ammonia, as N",GLO,0.844921,Air,(unspecified)
1403,Ammonium carbonate,GLO,0.249229,Air,(unspecified)
1637,Ammonium nitrate,GLO,0.451761,Air,(unspecified)
1871,Nitrate,GLO,0.291579,Air,(unspecified)
2105,Nitric oxide,GLO,0.602391,Air,(unspecified)
2339,Nitrite,GLO,0.392903,Air,(unspecified)
2573,Nitrogen dioxide,GLO,0.392903,Air,(unspecified)
2807,Nitrogen oxides,GLO,0.393038,Air,(unspecified)
3041,Sulfate,GLO,0.666933,Air,(unspecified)


In [25]:
cfs_midpoint = cfs_midpoint.drop(cfs_midpoint.loc[cfs_midpoint.loc[:, 'Region code'].isin(['GLO'])].index)

### Reorganising the dataframe of the cfs

Extracting the biosphere flows

In [26]:
unique_rows = cfs_midpoint.drop_duplicates(subset=['Elem flow', 'Compartment', 'Sub-compartment'])
# Creating a dataframe with unique column name with the following format: "Elem flow";Compartment::sub-compartment" 

biosphere_flow = {
    f"{row['Elem flow']};{row['Compartment']}::{row['Sub-compartment']}": {
        row['Elem flow']: [row['Compartment'].lower(), row['Sub-compartment'].lower()]}
    for _, row in unique_rows.iterrows()
}

for key in biosphere_flow.keys():
    for comp in biosphere_flow[key].values():
        if "(unspecified)" in comp:
            tupple_comp = (comp[0],)
            for flow in biosphere_flow[key].keys():
                biosphere_flow[key][flow] = tupple_comp

Creating a dataframe with unique column name with the following format: "Elem flow";Compartment::sub-compartment" 

In [27]:
full_name_flow = unique_rows.apply(
    lambda row: f"{row['Elem flow']};{row['Compartment']}::{row['Sub-compartment']}",
    axis=1
)

Creating a new column with the names of the flow with the following format: "Elem flow";Compartment::sub-compartment"

In [28]:
cfs_midpoint.loc[:, "full name flow"] = cfs_midpoint.apply(
    lambda row: f"{row['Elem flow']};{row['Compartment']}::{row['Sub-compartment']}",
    axis=1
)

In [29]:
cfs_midpoint = cfs_midpoint.drop(labels = ["Elem flow","Compartment","Sub-compartment"], axis = 1)

We must have one country per line of the dataframe and all the different caracterisation factors on the same line. We must pivot the cfs_midpoint dataframe.

In [30]:
pivot_df = cfs_midpoint.pivot(
    index="Region code",          
    columns="full name flow",     
    values="CF value"             
)

In [31]:
pivot_df = pivot_df.reset_index()

In [32]:
pivot_df

full name flow,Region code,"Ammonia, as N;Air::(unspecified)",Ammonia;Air::(unspecified),Ammonium carbonate;Air::(unspecified),Ammonium nitrate;Air::(unspecified),"Ammonium, ion;Air::(unspecified)",Nitrate;Air::(unspecified),Nitric oxide;Air::(unspecified),Nitrite;Air::(unspecified),Nitrogen dioxide;Air::(unspecified),Nitrogen oxides;Air::(unspecified),Sulfate;Air::(unspecified),Sulfur dioxide;Air::(unspecified),Sulfur trioxide;Air::(unspecified),Sulfuric acid;Air::(unspecified)
0,AD,0.219037,0.180141,0.06461,0.118216,0.168643,0.0763,0.157632,0.102814,0.102814,0.102849,0.203056,0.304463,0.243631,0.198859
1,AE,0.125079,0.102868,0.036895,0.022574,0.096302,0.01457,0.0301,0.019633,0.019633,0.019639,0.030208,0.045293,0.036244,0.029583
2,AF,0.23517,0.19341,0.069369,0.098253,0.181064,0.063415,0.131013,0.085452,0.085452,0.085481,0.152026,0.227948,0.182404,0.148883
3,AG,0.220952,0.181716,0.065175,0.051328,0.170117,0.033128,0.068442,0.04464,0.04464,0.044656,0.086642,0.129912,0.103955,0.084851
4,AI,0.244883,0.201398,0.072234,0.065246,0.188542,0.042112,0.087001,0.056745,0.056745,0.056765,0.089545,0.134264,0.107438,0.087694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,YE,0.08136,0.066913,0.023999,0.013759,0.062642,0.008881,0.018347,0.011967,0.011967,0.011971,0.017784,0.026665,0.021337,0.017416
223,YT,0.025805,0.021223,0.007612,0.010306,0.019868,0.006652,0.013742,0.008963,0.008963,0.008966,0.011895,0.017836,0.014272,0.011649
224,ZA,0.02255,0.018546,0.006652,0.012443,0.017362,0.008031,0.016592,0.010822,0.010822,0.010825,0.021141,0.031699,0.025366,0.020704
225,ZM,0.021698,0.017845,0.0064,0.007423,0.016706,0.004791,0.009898,0.006456,0.006456,0.006458,0.017058,0.025577,0.020467,0.016706


In [33]:
cfs_midpoint = pivot_df

### Building a vectorial map of the cfs at the country scale

Download the vectorial map of the countries from Ecoinvent

In [34]:
from bw2data.utils import download_file

In [35]:
countries_fp = download_file("countries.gpkg","regional",url="https://geography.ecoinvent.org/files/",)

Opening the geopackage

In [36]:
countries_gpkg = gp.read_file(countries_fp)

Removing unnecessary columns

In [37]:
countries_gpkg = countries_gpkg.drop(labels=["id","gid","collection","name","uncode","longitude","isothreelettercode","unsubregion","latitude","unregion","uuid","shortname"],axis = 1)

In [38]:
countries_gpkg = countries_gpkg.dropna(subset=['isotwolettercode'])

Adding the regionalized CFs at the country scale of freshwater acidification in the countries vectorial map

In [39]:
fwa_gpkg = cfs_midpoint

In [40]:
#Ensure that both join columns are of the same type (str)
countries_gpkg['isotwolettercode'] = countries_gpkg['isotwolettercode'].astype(str)
fwa_gpkg['Region code'] = fwa_gpkg['Region code'].astype(str)

In [41]:
#A dictionary is created to map ISO codes to geometries.
geometry_map = countries_gpkg.set_index('isotwolettercode')['geometry'].to_dict()

In [42]:
#Add the geometry column to fwa_gpkg using the dictionary
fwa_gpkg['geometry'] = fwa_gpkg['Region code'].map(geometry_map)

In [43]:
#We convert fwa_gpkg to GeoDataFrame
fwa_gpkg = gp.GeoDataFrame(fwa_gpkg, geometry='geometry', crs=countries_gpkg.crs)

Make sure that the columns of fwa_gpkg are float64. Else the import of CFs will not work

In [44]:
fwa_gpkg.dtypes

full name flow
Region code                                object
Ammonia, as N;Air::(unspecified)           object
Ammonia;Air::(unspecified)                 object
Ammonium carbonate;Air::(unspecified)      object
Ammonium nitrate;Air::(unspecified)        object
Ammonium, ion;Air::(unspecified)           object
Nitrate;Air::(unspecified)                 object
Nitric oxide;Air::(unspecified)            object
Nitrite;Air::(unspecified)                 object
Nitrogen dioxide;Air::(unspecified)        object
Nitrogen oxides;Air::(unspecified)         object
Sulfate;Air::(unspecified)                 object
Sulfur dioxide;Air::(unspecified)          object
Sulfur trioxide;Air::(unspecified)         object
Sulfuric acid;Air::(unspecified)           object
geometry                                 geometry
dtype: object

In [45]:
for key in biosphere_flow.keys():
    fwa_gpkg[key] = fwa_gpkg[key].astype("float64")

In [46]:
fwa_gpkg.dtypes

full name flow
Region code                                object
Ammonia, as N;Air::(unspecified)          float64
Ammonia;Air::(unspecified)                float64
Ammonium carbonate;Air::(unspecified)     float64
Ammonium nitrate;Air::(unspecified)       float64
Ammonium, ion;Air::(unspecified)          float64
Nitrate;Air::(unspecified)                float64
Nitric oxide;Air::(unspecified)           float64
Nitrite;Air::(unspecified)                float64
Nitrogen dioxide;Air::(unspecified)       float64
Nitrogen oxides;Air::(unspecified)        float64
Sulfate;Air::(unspecified)                float64
Sulfur dioxide;Air::(unspecified)         float64
Sulfur trioxide;Air::(unspecified)        float64
Sulfuric acid;Air::(unspecified)          float64
geometry                                 geometry
dtype: object

We save the GeoDataFrame in a folder of the computer

In [47]:
fp = "C:\\Users\\rogyn\\PycharmProjects\\tutorial_brightway_regional_bw2\\Data\\Freshwater_acidification_country.gpkg"
fwa_gpkg.to_file(fp)

We check how fwa_gpkg looks

In [48]:
gp.read_file(fp)

,Region code,"Ammonia, as N;Air::(unspecified)",Ammonia;Air::(unspecified),Ammonium carbonate;Air::(unspecified),Ammonium nitrate;Air::(unspecified),"Ammonium, ion;Air::(unspecified)",Nitrate;Air::(unspecified),Nitric oxide;Air::(unspecified),Nitrite;Air::(unspecified),Nitrogen dioxide;Air::(unspecified),Nitrogen oxides;Air::(unspecified),Sulfate;Air::(unspecified),Sulfur dioxide;Air::(unspecified),Sulfur trioxide;Air::(unspecified),Sulfuric acid;Air::(unspecified),geometry
0,AD,0.219037,0.180141,0.064610,0.118216,0.168643,0.076300,0.157632,0.102814,0.102814,0.102849,0.203056,0.304463,0.243631,0.198859,"MULTIPOLYGON (((1.53885 42.44565, 1.53451 42.4..."
1,AE,0.125079,0.102868,0.036895,0.022574,0.096302,0.014570,0.030100,0.019633,0.019633,0.019639,0.030208,0.045293,0.036244,0.029583,"MULTIPOLYGON (((55.78402 24.58633, 55.76996 24..."
2,AF,0.235170,0.193410,0.069369,0.098253,0.181064,0.063415,0.131013,0.085452,0.085452,0.085481,0.152026,0.227948,0.182404,0.148883,"MULTIPOLYGON (((62.19629 29.47492, 62.11298 29..."
3,AG,0.220952,0.181716,0.065175,0.051328,0.170117,0.033128,0.068442,0.044640,0.044640,0.044656,0.086642,0.129912,0.103955,0.084851,"MULTIPOLYGON (((-62.34292 16.93837, -62.34306 ..."
4,AI,0.244883,0.201398,0.072234,0.065246,0.188542,0.042112,0.087001,0.056745,0.056745,0.056765,0.089545,0.134264,0.107438,0.087694,"MULTIPOLYGON (((-63.16784 18.16934, -63.14208 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,YE,0.081360,0.066913,0.023999,0.013759,0.062642,0.008881,0.018347,0.011967,0.011967,0.011971,0.017784,0.026665,0.021337,0.017416,"MULTIPOLYGON (((43.44744 12.63866, 43.4358 12...."
223,YT,0.025805,0.021223,0.007612,0.010306,0.019868,0.006652,0.013742,0.008963,0.008963,0.008966,0.011895,0.017836,0.014272,0.011649,"MULTIPOLYGON (((45.21762 -12.73406, 45.22478 -..."
224,ZA,0.022550,0.018546,0.006652,0.012443,0.017362,0.008031,0.016592,0.010822,0.010822,0.010825,0.021141,0.031699,0.025366,0.020704,"MULTIPOLYGON (((30.12338 -31.16155, 30.09506 -..."
225,ZM,0.021698,0.017845,0.006400,0.007423,0.016706,0.004791,0.009898,0.006456,0.006456,0.006458,0.017058,0.025577,0.020467,0.016706,"MULTIPOLYGON (((28.89889 -15.99546, 28.87718 -..."


## Creating a geocollection of the impact map

A ``geocollection`` is a dictionary of metadata containing :
- the filepath of an impact map
- the field (e.g. country name) : the name of the column of the cfs maps that correspond to a spatial section. The regionalized cfs will be linked to this spatial section.
- the kind (e.g. vector or raster)
- the hashing.

In [49]:
br.geocollections["Freshwater-acidification-country"] = {"filepath":fp,
                                                      "field":"Region code",
                                                      "kind" : "vector",
                                                     }

## Importing the regionalized CFs in brightway

A ``Method`` can have both site-generic and regionalized characterization factors

In [50]:
gdf = gp.read_file(br.geocollections["Freshwater-acidification-country"]["filepath"])

In [51]:
bio = bd.Database('biosphere3') 

Creating the flow_mapping

In [52]:
flow_mapping = {}
for key in biosphere_flow.keys():
    for flow in biosphere_flow[key].keys():
        flow_mapping[key] = [act.key for act in bio if flow == act["name"] and biosphere_flow[key][flow] == act["categories"]]
        if flow_mapping[key] == list(): #deleting flows for IW+ that are not in the biosphere of bw2
            del flow_mapping[key]

Creating the list of global characterisation factors

In [53]:
glo_flow = []
for _,features in cfs_glo.iterrows():
    if f"{features['Elem flow']};{features['Compartment']}::{features['Sub-compartment']}" in flow_mapping.keys():
        glo_flow.append((flow_mapping[f"{features['Elem flow']};{features['Compartment']}::{features['Sub-compartment']}"][0],float(features["CF value"]),"GLO"))

Choose a name for the method in the following method tupple and register it

In [54]:
method_tuple = ('IMPACT World+ Midpoint 2.1 for ecoinvent v3.10, regionalized at the country scale', 'Midpoint','freshwater acidification')

In [55]:
bd.Method(method_tuple).register()

Importing the regionalized CFs

In [56]:
br.import_regionalized_cfs(geocollection = "Freshwater-acidification-country",
                           method_tuple = method_tuple,
                           mapping = flow_mapping,
                           global_cfs = glo_flow)

Adding metadata

In [57]:
bd.Method(method_tuple).metadata["unit"]="kg SO2 eq"